In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [44]:
data = pd.read_csv('data/train.csv')

In [64]:
data.tail()

,City,City Group,Type,P1,P2,P3,P4,P5,P6,P7,...,P29,P30,P31,P32,P33,P34,P35,P36,P37,revenue
132,Trabzon,Other,FC,2,3.0,3.0,5.0,4,2,4,...,3.0,0,0,0,0,0,0,0,0,5787594.0
133,İzmir,Big Cities,FC,4,5.0,4.0,4.0,2,3,5,...,3.0,0,0,0,0,0,0,0,0,9262754.0
134,Kayseri,Other,FC,3,4.0,4.0,4.0,2,3,5,...,3.0,0,0,0,0,0,0,0,0,2544857.0
135,İstanbul,Big Cities,FC,4,5.0,4.0,5.0,2,2,5,...,3.0,0,0,0,0,0,0,0,0,7217634.0
136,İstanbul,Big Cities,FC,4,5.0,3.0,5.0,2,2,5,...,3.0,0,0,0,0,0,0,0,0,6363241.0


In [46]:
data.shape

(137, 43)

In [47]:
data['City Group'].value_counts()

Big Cities    78
Other         59
Name: City Group, dtype: int64

In [48]:
#One Hot Encoding : City , Type
#Ordinal Ecnoder : City Group
#P1 -> P37 Normalization
#Delete : Id , Open Date

In [65]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

In [53]:
dataframe.drop(['Open Date', 'Id'], axis = 1, inplace = True)

In [63]:
column_trans = make_column_transformer((OneHotEncoder(), ['City', 'Type']), remainder='passthrough')
column_trans.fit_transform(data)

array([[0.0, 0.0, 0.0, ..., 3, 4, 5653753.0],
       [0.0, 0.0, 0.0, ..., 0, 0, 6923131.0],
       [0.0, 0.0, 0.0, ..., 0, 0, 2055379.0],
       ...,
       [0.0, 0.0, 0.0, ..., 0, 0, 2544857.0],
       [0.0, 0.0, 0.0, ..., 0, 0, 7217634.0],
       [0.0, 0.0, 0.0, ..., 0, 0, 6363241.0]], dtype=object)

In [67]:
ord_enc = OrdinalEncoder()
ord_enc.fit_transform(data[['City Group']])

array([[0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],

In [68]:
data

,City,City Group,Type,P1,P2,P3,P4,P5,P6,P7,...,P29,P30,P31,P32,P33,P34,P35,P36,P37,revenue
0,İstanbul,Big Cities,IL,4,5.0,4.0,4.0,2,2,5,...,3.0,5,3,4,5,5,4,3,4,5653753.0
1,Ankara,Big Cities,FC,4,5.0,4.0,4.0,1,2,5,...,3.0,0,0,0,0,0,0,0,0,6923131.0
2,Diyarbakır,Other,IL,2,4.0,2.0,5.0,2,3,5,...,3.0,0,0,0,0,0,0,0,0,2055379.0
3,Tokat,Other,IL,6,4.5,6.0,6.0,4,4,10,...,7.5,25,12,10,6,18,12,12,6,2675511.0
4,Gaziantep,Other,IL,3,4.0,3.0,4.0,2,2,5,...,3.0,5,1,3,2,3,4,3,3,4316715.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,Trabzon,Other,FC,2,3.0,3.0,5.0,4,2,4,...,3.0,0,0,0,0,0,0,0,0,5787594.0
133,İzmir,Big Cities,FC,4,5.0,4.0,4.0,2,3,5,...,3.0,0,0,0,0,0,0,0,0,9262754.0
134,Kayseri,Other,FC,3,4.0,4.0,4.0,2,3,5,...,3.0,0,0,0,0,0,0,0,0,2544857.0
135,İstanbul,Big Cities,FC,4,5.0,4.0,5.0,2,2,5,...,3.0,0,0,0,0,0,0,0,0,7217634.0
